In [1]:
#import dependencies 
import psycopg2
import pandas as pd

In [2]:
from getpass import getpass
password = getpass("Enter database password")

param_dic = {
    "host" : "localhost",
    "database" : "BRFSSAnalysis",
    "user" : "postgres",
    "password" : password
}

Enter database password········


In [3]:
def connect(params_dic):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    SELECT * from question_info
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [7]:
# Connect to the database
conn = connect(param_dic)
column_names = ["id", "var_name", "label", "text"]
# Execute the "SELECT *" query to save question_info as a datafram
question_info_df = postgresql_to_dataframe(conn, "select * from question_info", column_names)
question_info_df.head()

Connecting to the PostgreSQL database...
Connection successful


,id,var_name,label,text
0,1,_STATE,State FIPS Code,State FIPS Code
1,2,FMONTH,File Month,File Month
2,3,IDATE,Interview Date,Interview Date
3,4,IMONTH,Interview Month,Interview Month
4,5,IDAY,Interview Day,Interview Day


In [31]:
# Connect to the database and repeat process for question_values
conn = connect(param_dic)
column_names = ["id", "question_id", "label", "value", "value_end"]
# Execute the "SELECT *" query to save question_info as a datafram
answer_info_df = postgresql_to_dataframe(conn, "select * from question_values", column_names)
answer_info_df.head()

Connecting to the PostgreSQL database...
Connection successful


,id,question_id,label,value,value_end
0,1,1,Alabama,1,None
1,2,1,Alaska,2,None
2,3,1,Arizona,4,None
3,4,1,Arkansas,5,None
4,5,1,California,6,None


In [28]:
#determing which columns of user answers (aka relevant answers) to bring over
a = question_info_df['label'].values.tolist()
b = question_info_df['var_name'].values.tolist()

In [26]:
zip_iterator = zip(b, a)
a_dictionary = dict(zip_iterator)

In [27]:
for key, value in a_dictionary.items():
    print(key, ' : ', value)

_STATE  :  State FIPS Code
FMONTH  :  File Month
IDATE  :  Interview Date
IMONTH  :  Interview Month
IDAY  :  Interview Day
IYEAR  :  Interview Year
DISPCODE  :  Final Disposition
SEQNO  :  Annual Sequence Number
_PSU  :  Primary Sampling Unit
CTELENM1  :  Correct telephone number?
PVTRESD1  :  Private Residence?
COLGHOUS  :  Do you live in college housing?
STATERE1  :  Resident of State
CELPHONE  :  Cellular Telephone
LADULT1  :  Are you 18 years of age or older?
COLGSEX  :  Are you male or female?
NUMADULT  :  Number of Adults in Household
LANDSEX  :  Are you male or female?
NUMMEN  :  Number of Adult men in Household
NUMWOMEN  :  Number of Adult women in Household
RESPSLCT  :  Respondent selection
SAFETIME  :  Safe time to talk
CTELNUM1  :  Correct Phone Number?
CELLFON5  :  Is this a cell phone?
CADULT1  :  Are you 18 years of age or older?
CELLSEX  :  Are you male or female?
PVTRESD3  :  Do you live in a private residence?
CCLGHOUS  :  Do you live in college housing?
CSTATE1  :  D

In [42]:
#select columns to include from list above for health behaviour features predicting general, mental and physical health
columns = ["GENHLTH", "PHYSHLTH", "MENTHLTH", "EXRACT11", "PAFREQ1_", "_MINAC11", "ACTIN12_", "EXRACT21", "PAFREQ2_", "_MINAC21", "ACTIN22_",  "STRFREQ_", "PA2MIN_", "_METSTAT", "_URBSTAT", "HTIN4", "WTKG3", "_BMI5", "_SMOKER3", "_DRNKWK1", "FC601_", "FTJUDA2_",  "GRENDA1_", "VEGEDA2_", "POTADA1_", "FRNCHDA_", "_FRUTSU1", "_FRUTSU1", "_VEGESU1"]                 

In [43]:
# Connect to the database and repeat process for user_answers: limit of 500
conn = connect(param_dic)
column_names = columns
# Execute the "SELECT *" query to save question_info as a datafram
health_behaviour_df = postgresql_to_dataframe(conn, f"select GENHLTH, PHYSHLTH, MENTHLTH, EXRACT11, PAFREQ1_, _MINAC11, ACTIN12_, EXRACT21, PAFREQ2_, _MINAC21, ACTIN22_,  STRFREQ_, PA2MIN_, _METSTAT, _URBSTAT, HTIN4, WTKG3, _BMI5, _SMOKER3, _DRNKWK1, FC601_, FTJUDA2_,  GRENDA1_, VEGEDA2_, POTADA1_, FRNCHDA_, _FRUTSU1, _FRUTSU1, _VEGESU1 from user_answers limit 500", column_names)


Connecting to the PostgreSQL database...
Connection successful


In [45]:
health_behaviour_df.head()

,GENHLTH,PHYSHLTH,MENTHLTH,EXRACT11,PAFREQ1_,_MINAC11,ACTIN12_,EXRACT21,PAFREQ2_,_MINAC21,...,_DRNKWK1,FC601_,FTJUDA2_,GRENDA1_,VEGEDA2_,POTADA1_,FRNCHDA_,_FRUTSU1,_FRUTSU1,_VEGESU1
0,1.0,88.0,88.0,67.0,3000.0,0.0,0.0,37.0,1400.0,42.0,...,400.0,614.0,14.0,71.0,43.0,29.0,57.0,81.0,81.0,200.0
1,4.0,3.0,30.0,67.0,3000.0,0.0,0.0,64.0,2000.0,60.0,...,1400.0,557.0,43.0,14.0,29.0,0.0,0.0,86.0,86.0,43.0
2,4.0,15.0,88.0,64.0,3500.0,105.0,1.0,88.0,None,0.0,...,0.0,387.0,0.0,14.0,10.0,0.0,3.0,100.0,100.0,27.0
3,2.0,30.0,30.0,None,None,None,None,None,None,None,...,0.0,480.0,None,None,None,None,None,None,None,None
4,2.0,2.0,88.0,64.0,2000.0,60.0,1.0,18.0,1000.0,None,...,0.0,398.0,0.0,29.0,71.0,29.0,29.0,71.0,71.0,158.0


In [46]:
print(health_behaviour_df.isnull().sum())

GENHLTH       0
PHYSHLTH      0
MENTHLTH      0
EXRACT11    157
PAFREQ1_    162
_MINAC11    170
ACTIN12_    163
EXRACT21    161
PAFREQ2_    272
_MINAC21    177
ACTIN22_    174
STRFREQ_     37
PA2MIN_     167
_METSTAT      0
_URBSTAT      0
HTIN4        21
WTKG3        45
_BMI5        48
_SMOKER3      0
_DRNKWK1      0
FC601_        0
FTJUDA2_     36
GRENDA1_     42
VEGEDA2_     45
POTADA1_     46
FRNCHDA_     39
_FRUTSU1     47
_FRUTSU1     47
_VEGESU1     58
dtype: int64


In [ ]:
#TODO: go through features and correct poorly coded values (e.g. 9999 = no answer)
#clean up DF for null values
#feature analysis to determine best predictors of mental and physical health

In [ ]:
#TODO: make a dataframe that contains the demographic information of the sample and their health outcomes 
#Draw up summary of sample

In [ ]:
#map health outcomes accross states

In [ ]:
#create a large df with demographic and health behaviour info and create model that can predict health outcomes